
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# 5 - Deploying a Pipeline to Production

In this demonstration, we will begin by adding an additional data source to our pipeline and performing a join with our streaming tables. Then, we will focus on productionalizing the pipeline by adding comments and table properties to the objects we create, scheduling the pipeline, and creating an event log to monitor the pipeline.

### Learning Objectives

By the end of this lesson, you will be able to:
- Apply the appropriate comment syntax and table properties to pipeline objects to enhance readability.
- Demonstrate how to perform a join between two streaming tables using a materialized view to optimize data processing.
- Execute the scheduling of a pipeline using trigger or continuous modes to ensure timely processing.
- Explore the event log to monitor a production Lakeflow Declarative Pipeline.

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

    - In the drop-down, select **More**.

    - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course.

This cell will also reset your `/Volumes/dbacademy/ops/labuser/` volume with the JSON files to the starting point, with one JSON file in each volume.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically create and reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-5

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Schema labuser11058730_1754017152.1_bronze_db already exists. No action taken.
Schema labuser11058730_1754017152.2_silver_db already exists. No action taken.
Schema labuser11058730_1754017152.3_gold_db already exists. No action taken.
----------------------------------------------------------------------------------------
Directory /Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com/customers already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com/orders already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com/status already exists. No action taken.
----------------------------------------------------------------------------------------


Searching for files in /Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com/customers/ volume to delete prior to creating files...
Deleting file: /Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com/customers/00.json

Searc

Schemas are available, lab check passed: ['1_bronze_db', '2_silver_db', '3_gold_db'].


Your catalog name variable reference: DA.catalog_name:,
"Variable reference to your source files (Python - DA.paths.working_dir, SQL - DA.paths_working_dir):",


## B. Explore the Orders and Status JSON Files

1. Explore the raw data located in the `/Volumes/dbacademy/ops/our-lab-user/orders/` volume. This is the data we have been working with throughout the course demonstrations.

   Run the cell below to view the results. Notice that the orders JSON file(s) contains information about when each order was placed.

In [0]:
SELECT *
FROM read_files(
  DA.paths_working_dir || '/orders/',
  format => 'JSON'
)
LIMIT 10;

customer_id,notifications,order_id,order_timestamp,_rescued_data
23094,Y,75123,1640392092,null
23457,N,75124,1640392500,null
23564,Y,75125,1640394862,null
23392,N,75126,1640396067,null
23101,Y,75127,1640399066,null
23466,N,75128,1640404853,null
23834,Y,75129,1640407272,null
23852,Y,75130,1640419989,null
23483,Y,75131,1640422131,null
23821,N,75132,1640423697,null


2. Explore the **status** raw data located in the `/Volumes/dbacademy/ops/your-lab-user/status/` volume and filter for the specific **order_id** *75123*.

   Run the cell below to view the results. Notice that the status JSON file(s) contain **order_status** information for each order.  
   
   **NOTE:** The **order_status** can include multiple rows per order and may be any of the following:
   
   - on the way  
   - canceled  
   - return canceled  
   - reported shipping error  
   - delivered  
   - return processed  
   - return picked up  
   - placed  
   - preparing  
   - return requested


In [0]:
SELECT *
FROM read_files(
  DA.paths_working_dir || '/status/',
  format => 'JSON'
)
WHERE order_id = 75123;

order_id,order_status,status_timestamp,_rescued_data
75123,placed,1640392092,null
75123,preparing,1640733966,null
75123,on the way,1640549318,null
75123,delivered,1640604626,null


3. One of our objectives is to join the **orders** data with the order **status** data.  

    The query below demonstrates what the result of the final join in the Lakeflow Declarative Pipeline will look like after the data has been incrementally ingested and cleaned when we create the pipeline. Run the cell and review the output.

    Notice that after joining the tables, we can see each **order_id** along with its original **order_timestamp** and the **order_status** at specific points in time.

**NOTE:** The data used in this demo is artificially generated, so the **order_status_timestamps** may not reflect realistic timing.

In [0]:
WITH orders AS (
  SELECT *
  FROM read_files(
        DA.paths_working_dir || '/orders/',
        format => 'JSON'
  )
),
status AS (
  SELECT *
  FROM read_files(
        DA.paths_working_dir || '/status/',
        format => 'JSON'
  )
)
-- Join the views to get the order history with status
SELECT
  orders.order_id,
  timestamp(orders.order_timestamp) AS order_timestamp,
  status.order_status,
  timestamp(status.status_timestamp) AS order_status_timestamp
FROM orders
  INNER JOIN status 
  ON orders.order_id = status.order_id
ORDER BY order_id, order_status_timestamp;

order_id,order_timestamp,order_status,order_status_timestamp
75123,2021-12-25T00:28:12Z,placed,2021-12-25T00:28:12Z
75123,2021-12-25T00:28:12Z,on the way,2021-12-26T20:08:38Z
75123,2021-12-25T00:28:12Z,delivered,2021-12-27T11:30:26Z
75123,2021-12-25T00:28:12Z,preparing,2021-12-28T23:26:06Z
75124,2021-12-25T00:35:00Z,placed,2021-12-25T00:35:00Z
75124,2021-12-25T00:35:00Z,preparing,2021-12-27T13:20:27Z
75124,2021-12-25T00:35:00Z,on the way,2021-12-27T14:11:48Z
75124,2021-12-25T00:35:00Z,delivered,2021-12-27T15:53:42Z
75125,2021-12-25T01:14:22Z,placed,2021-12-25T01:14:22Z
75125,2021-12-25T01:14:22Z,delivered,2021-12-26T20:37:20Z


## C. Putting a Pipeline in Production

This course includes a complete Lakeflow Declarative Pipeline project that has already been created.  In this section, you'll explore the Lakeflow Declarative Pipeline modify its settings for production use.


1. The screenshot below shows what the final Lakeflow Declarative Pipeline will look like when ingesting a single JSON file from the data sources:  
![Final Demo 6 Pipeline](./Includes/images/demo6_pipeline_image.png)

    **Note:** Depending on the number of files you've ingested, the row count may vary.

2. Run the cell below to create your starter Lakeflow Declarative Pipeline for this demonstration. The pipeline will set the following for you:
    - Your default catalog: `labuser`
    - Your configuration parameter: `source` = `/Volumes/dbacademy/ops/your-labuser-name`

    **NOTE:** If the pipeline already exists, an error will be returned. In that case, you'll need to delete the existing pipeline and rerun this cell.

    To delete the pipeline:

    a. Select **Jobs and Pipelines** from the far-left navigation bar.  

    b. Find the pipeline you want to delete.  

    c. Click the three-dot menu ![ellipsis icon](./Includes/images/ellipsis_icon.png).  

    d. Select **Delete**.

**NOTE:**  The `create_declarative_pipeline` function is a custom function built for this course to create the sample pipeline using the Databricks REST API. This avoids manually creating the pipeline and referencing the pipeline assets.

In [0]:
%python
create_declarative_pipeline(pipeline_name=f'5 - Deploying a Pipeline to Production Project - {DA.catalog_name}', 
                            root_path_folder_name='5 - Deploying a Pipeline to Production Project',
                            catalog_name = DA.catalog_name,
                            schema_name = 'default',
                            source_folder_names=['orders', 'status'],
                            configuration = {'source':DA.paths.working_dir})

Creating the Lakeflow Declarative Pipeline '5 - Deploying a Pipeline to Production Project - labuser11058730_1754017152'...
Root folder path: /Workspace/Users/labuser11058730_1754017152@vocareum.com/build-data-pipelines-with-lakeflow-declarative-pipelines-3.0.2/Build Data Pipelines with Lakeflow Declarative Pipelines/5 - Deploying a Pipeline to Production Project
Source folder path(s): [{'glob': {'include': '/Workspace/Users/labuser11058730_1754017152@vocareum.com/build-data-pipelines-with-lakeflow-declarative-pipelines-3.0.2/Build Data Pipelines with Lakeflow Declarative Pipelines/5 - Deploying a Pipeline to Production Project/orders/**'}}, {'glob': {'include': '/Workspace/Users/labuser11058730_1754017152@vocareum.com/build-data-pipelines-with-lakeflow-declarative-pipelines-3.0.2/Build Data Pipelines with Lakeflow Declarative Pipelines/5 - Deploying a Pipeline to Production Project/status/**'}}]

Lakeflow Declarative Pipeline Creation '5 - Deploying a Pipeline to Production Project - 

3. Complete the following steps to open the starter Lakeflow Declarative Pipeline project for this demonstration:

   a. Click the folder icon ![Folder](./Includes/images/folder_icon.png) in the left navigation panel.

   b. In the **Build Data Pipelines with Lakeflow Declarative Pipelines** folder, find the **5 - Deploying a Pipeline to Production Project** folder. 
   
   c. Right-click and select **Open in a new tab**.

   d. In the new tab you should see three folders: **explorations**, **orders**, and **status** (plus the extra **python_excluded** folder that contains the Python version). 

   e. Continue to step 4 and 5 below.

#### IMPORTANT
   **NOTE:** If you open the **orders_pipeline.sql** file and it does not open up the pipeline editor, that is because that folder is not associated with a pipeline. Please make sure to run the previous cell to associate the folder with the pipeline and try again.

   **WARNING:** If you get the following warning when opening the **orders_pipeline.sql** file: 

   ```pipeline you are trying to access does not exist or is inaccessible. Please verify the pipeline ID, request access or detach this file from the pipeline.``` 

   Simply refresh the page and/or reselect the notebook.

##### Explore the code in the `orders/orders_pipeline.sql` file

4. In the new tab select the **orders** folder. It contains the same **orders_pipeline.sql** pipeline you've been working with.  Follow the instructional comments in the file to proceed.



##### Explore the code in the `status/status_pipeline` notebook

5. After reviewing the **orders_pipeline.sql** file, you'll be directed to explore the **status/status_pipeline.sql** notebook. This notebook processes new data and adds it to the pipeline. Follow the instructions provided in the notebook's markdown cells.

    **NOTE:** The **status/status_pipeline.sql**  notebook will go through setting up the pipeline settings, scheduling and running the production pipeline.

## D. Land More Data to Your Data Source Volume

1. Run the cell below to add **4** more JSON files to your **orders** and **status** volume.

In [0]:
%python
copy_files(copy_from = '/Volumes/dbacademy_retail/v01/retail-pipeline/orders/stream_json', 
           copy_to = f'{DA.paths.working_dir}/orders', 
           n = 5)

copy_files(copy_from = '/Volumes/dbacademy_retail/v01/retail-pipeline/status/stream_json', 
           copy_to = f'{DA.paths.working_dir}/status', 
           n = 5)


----------------Loading files to user's volume: '/Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com/orders'----------------
File number 1 - 00.json is already in the source volume "/Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com/orders". Skipping file.
File number 2 - Copying file /Volumes/dbacademy_retail/v01/retail-pipeline/orders/stream_json/01.json --> /Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com/orders/01.json.
File number 3 - Copying file /Volumes/dbacademy_retail/v01/retail-pipeline/orders/stream_json/02.json --> /Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com/orders/02.json.
File number 4 - Copying file /Volumes/dbacademy_retail/v01/retail-pipeline/orders/stream_json/03.json --> /Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com/orders/03.json.
File number 5 - Copying file /Volumes/dbacademy_retail/v01/retail-pipeline/orders/stream_json/04.json --> /Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum

2. Complete the follow to confirm new JSON files were added to your volume:

   a. Select the **Catalog** icon ![Catalog Icon](./Includes/images/catalog_icon.png) in the left navigation bar.

   b. Expand the **dbacademy** catalog.

   c. Expand the **ops** schema.

   d. Expand your **labuser** volume. You should notice that your volume contains three folders:
   - **customers**
   - **orders**
   - **status**

   e. Expand the **orders** and **status** folders.

   f. Notice that we have landed new data in our data source for the pipeline. You should have a total of **5** JSON files in each.

3. Navigate back to your pipeline and select **Run pipeline** to process the new landed files.

## E. Introduction to the Pipeline Event Log (Advanced Topic)

After running your pipeline and successfully publishing the event log as a table named **event_log_demo_5** in your **labuser.default** schema (database), begin exploring the event log. 

Here we will quickly introduce the event log. **To process the event log you will need knowledge of parsing JSON formatted strings.**

  - [Monitor Lakeflow Declarative Pipelines](https://docs.databricks.com/aws/en/dlt/observability) documentation

**TROUBLESHOOT:** 
- **REQUIRED:** If you did not run the pipeline and publish the event log, the code below will not run. Please make sure to complete all steps before starting this section.

- **HIDDEN EVENT LOG:** By default, Lakeflow Declarative Pipelines writes the event log to a hidden Delta table in the default catalog and schema configured for the pipeline. While hidden, the table can still be queried by all sufficiently privileged users. By default, only the owner of the pipeline can query the event log table. By default, the name for the hidden event log is formatted as:  
  - `catalog.schema.event_log_{pipeline_id}` – where the pipeline ID is the system-assigned UUID with dashes replaced by underscores.  
  - [Query the Event Log](https://docs.databricks.com/aws/en/dlt/observability#query-the-event-log)

1. Complete the following steps to view the **labuser.default.event_log_demo_5** event log in your catalog:

   a. Select the catalog icon ![Catalog Icon](./Includes/images/catalog_icon.png) from the left navigation pane.

   b. Expand your **labuser** catalog.

   c. Expand the following schemas (databases):
      - **1_bronze_db**
      - **2_silver_db**
      - **3_gold_db**
      - **default**

   d. Notice the following:
      - In the **1_bronze_db**, **2_silver_db**, and **3_gold_db** schemas, the pipeline streaming tables and materialized views were created (they end with **demo5**).
      - In the **default** schema, the pipeline has published the event log as a table named **event_log_demo_5**.

**NOTE:** You might need to refresh the catalogs to view the streaming tables, materialized views, and event log.


2. Query your **labuser.default.event_log_demo_5** table to see what the event log looks like.

   Notice that it contains all events within the pipeline as **STRING** columns (typically JSON-formatted strings) or **STRUCT** columns. Databricks supports the `:` (colon) operator to parse JSON fields. See the [`:` operator documentation](https://docs.databricks.com/) for more details.

   The following table describes the event log schema. Some fields contain JSON data—such as the **details** field—which must be parsed to perform certain queries.

In [0]:
SELECT *
FROM default.event_log_demo_5;

id sequence origin timestamp message level maturity_level error details event_type 3bb55e50-6ea0-11f0-a268-aa11d67e9c2a List(List(execution, 1754029489000001), null) List(AWS, us-west-2, 3622490971720, null, 6ce1fdf2-b7dd-4ed3-844e-1d170666cadb, WORKSPACE, 5 - Deploying a Pipeline to Production Project - labuser11058730_1754017152, 0801-062429-la5rq55-v2n, e994a5ed-09c2-41af-90f3-9d8c34773039, null, null, null, null, null, null, e994a5ed-09c2-41af-90f3-9d8c34773039, null, null, null, null, null, null, null, null, null, null, null, null) 2025-08-01T06:24:52.405Z Update e994a5 is INITIALIZING. INFO STABLE null {"update_progress":{"state":"INITIALIZING"}} update_progress aa824e80-6e9e-11f0-b947-8acb90f2812d List(List(execution, 1754029489000003), 1754028819305001) List(AWS, us-west-2, 3622490971720, null, 6ce1fdf2-b7dd-4ed3-844e-1d170666cadb, WORKSPACE, 5 - Deploying a Pipeline to Production Project - labuser11058730_1754017152, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null) 2025-08-01T06:13:39.304Z User labuser11058730_1754017152@vocareum.com created pipeline. INFO STABLE null {"user_action":{"action":"CREATE","user_name":"labuser11058730_1754017152@vocareum.com","user_id":70949121692270,"request":{"create_request":{"id":"6ce1fdf2-b7dd-4ed3-844e-1d170666cadb","pipeline_type":"WORKSPACE","name":"5 - Deploying a Pipeline to Production Project - labuser11058730_1754017152","configuration":{"source":"/Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com"},"libraries":[{"glob":{"include":"/Workspace/Users/labuser11058730_1754017152@vocareum.com/build-data-pipelines-with-lakeflow-declarative-pipelines-3.0.2/Build Data Pipelines with Lakeflow Declarative Pipelines/5 - Deploying a Pipeline to Production Project/orders/**"}},{"glob":{"include":"/Workspace/Users/labuser11058730_1754017152@vocareum.com/build-data-pipelines-with-lakeflow-declarative-pipelines-3.0.2/Build Data Pipelines with Lakeflow Declarative Pipelines/5 - Deploying a Pipeline to Production Project/status/**"}}],"schema":"default","continuous":false,"development":true,"photon":true,"channel":"PREVIEW","catalog":"labuser11058730_1754017152","serverless":true,"root_path":"/Workspace/Users/labuser11058730_1754017152@vocareum.com/build-data-pipelines-with-lakeflow-declarative-pipelines-3.0.2/Build Data Pipelines with Lakeflow Declarative Pipelines/5 - Deploying a Pipeline to Production Project"}}}} user_action 1b0fd310-6ea0-11f0-b947-8acb90f2812d List(List(execution, 1754029489000004), 1754029437634001) List(AWS, us-west-2, 3622490971720, null, 6ce1fdf2-b7dd-4ed3-844e-1d170666cadb, WORKSPACE, 5 - Deploying a Pipeline to Production Project - labuser11058730_1754017152, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null) 2025-08-01T06:23:57.633Z User labuser11058730_1754017152@vocareum.com edited pipeline settings. INFO STABLE null {"user_action":{"action":"EDIT","user_name":"labuser11058730_1754017152@vocareum.com","user_id":70949121692270,"request":{"edit_request":{"id":"6ce1fdf2-b7dd-4ed3-844e-1d170666cadb","pipeline_type":"WORKSPACE","name":"5 - Deploying a Pipeline to Production Project - labuser11058730_1754017152","configuration":{"source":"/Volumes/dbacademy/ops/labuser11058730_1754017152@vocareum_com"},"libraries":[{"glob":{"include":"/Workspace/Users/labuser11058730_1754017152@vocareum.com/build-data-pipelines-with-lakeflow-declarative-pipelines-3.0.2/Build Data Pipelines with Lakeflow Declarative Pipelines/5 - Deploying a Pipeline to Production Project/orders/**"}},{"glob":{"include":"/Workspace/Users/labuser11058730_1754017152@vocareum.com/build-data-pipelines-with-lakeflow-declarative-pipelines-3.0.2/Build Data Pipelines with Lakeflow Declarative Pipelines/5 - Deploying a Pipeline to Production Project/status/**"}}],"schema":"default","continuous":false,"development":false,"photon":true,"channel":"PREVIEW"

| Field          | Description |
|----------------|-------------|
| `id`           | A unique identifier for the event log record. |
| `sequence`     | A JSON document containing metadata to identify and order events. |
| `origin`       | A JSON document containing metadata for the origin of the event, for example, the cloud provider, the cloud provider region, user_id, pipeline_id, or pipeline_type to show where the pipeline was created, either DBSQL or WORKSPACE. |
| `timestamp`    | The time the event was recorded. |
| `message`      | A human-readable message describing the event. |
| `level`        | The event type, for example, INFO, WARN, ERROR, or METRICS. |
| `maturity_level` | The stability of the event schema. The possible values are:<br><br>- **STABLE**: The schema is stable and will not change.<br>- **NULL**: The schema is stable and will not change. The value may be NULL if the record was created before the maturity_level field was added (release 2022.37).<br>- **EVOLVING**: The schema is not stable and may change.<br>- **DEPRECATED**: The schema is deprecated and the pipeline runtime may stop producing this event at any time. |
| `error`        | If an error occurred, details describing the error. |
| `details`      | A JSON document containing structured details of the event. This is the primary field used for analyzing events. |
| `event_type`   | The event type. |

**[Event Log Schema](https://docs.databricks.com/aws/en/dlt/observability#event-log-schema)**

3. The majority of the detailed information you will want from the event log is located in the **details** column, which is a JSON-formatted string. You will need to parse this column.

   You can find more information in the Databricks documentation on how to [query JSON strings](https://docs.databricks.com/aws/en/semi-structured/json).

   The code below will:

   - Return the **event_type** column.

   - Return the entire **details** JSON-formatted string.

   - Parse out the **flow_progress** values from the **details** JSON-formatted string, if they exist.
   
   - Parse out the **user_action** values from the **details** JSON-formatted string, if they exist.


In [0]:
SELECT
  id,
  event_type,
  details,
  details:flow_progress,
  details:user_action
FROM default.event_log_demo_5

id event_type details flow_progress user_action ccdb9250-6ea0-11f0-a268-aa11d67e9c2a update_progress {"update_progress":{"state":"INITIALIZING"}} null null cc133940-6ea0-11f0-a6b2-02a90fbeb9b7 user_action {"user_action":{"action":"START","user_name":"labuser11058730_1754017152@vocareum.com","user_id":70949121692270,"request":{"start_request":{"full_refresh":false,"validate_only":false,"explore_only":false,"development":true}}}} null {"action":"START","user_name":"labuser11058730_1754017152@vocareum.com","user_id":70949121692270,"request":{"start_request":{"full_refresh":false,"validate_only":false,"explore_only":false,"development":true}}} cc1e0eb0-6ea0-11f0-a6b2-02a90fbeb9b7 create_update {"create_update":{"cause":"USER_ACTION","config":{"id":"6ce1fdf2-b7dd-4ed3-844e-1d170666cadb","pipeline_type":"WORKSPACE","name":"5 - Deploying a Pipeline to Production Project - labuser11058730_1754017152","configuration":{"pipelines.acfsProcessorVersion":"1","pipelines.allowClearingTableComment":"true","pipelines.allowPrivateToNonPrivateAltersWithCustomSchemaPreview":"true","pipelines.alterExistingMvSt":"true","pipelines.alterViewsInHMS.enabled":"true","pipelines.alterableMetadataBehavior":"preserve","pipelines.analysis.allowSqlFlowWithConfParallelResolution":"true","pipelines.analysis.maybeResolveFlowsParallely":"true","pipelines.analysis.maybeResolvePureSQLPipelinesParallely":"true","pipelines.analysis.pruneUnresolvedFromJsonDependents":"true","pipelines.applyChanges.Type2IgnoreNullPersistentStateChange":"false","pipelines.applyChanges.applyChangesMultiflowIgnoreNullOffFix.enabled":"false","pipelines.applyChanges.applyChangesMultiflowLatestDeleteVersion.enabled":"false","pipelines.applyChanges.dynamicFilePruning.enabled":"true","pipelines.applyChanges.dynamicPartitionPruning.enabled":"true","pipelines.applyChanges.viewMetrics.enabled":"true","pipelines.applyChangesFromSnapshot.analyzeLambdaSourceWithoutPipelineContext":"true","pipelines.applyChangesFromSnapshot.castSnapshotToTargetSchema":"true","pipelines.applyChangesFromSnapshot.stateStoreFormat":"proto","pipelines.attachEnzymeExpectationsUsingTransformDF":"true","pipelines.attachExpectationWithFunctionResult":"true","pipelines.attachExpectationsUsingTransformDF":"true","pipelines.autoscaling.maxDecommissioningTimeoutInSecondsDefaultValue":"14400","pipelines.blockDuplicatedUseOfFromJsonSchemaLocationKey":"true","pipelines.cdc.enableCdcSnapshotProgress":"true","pipelines.cdc.enableGatewayErrorPropagation":"true","pipelines.cdc.skipCdcSnapshotFlowAfterCommitted":"true","pipelines.cdc.stagingDataGCDryRunMode":"false","pipelines.cdc.stagingDataRetentionDays":"25","pipelines.cdcApplier.enableCommunicationVersioning":"true","pipelines.cdcApplier.useUpstreamCommunicationLibrary":"true","pipelines.cdcApplierEnableColumnUOID":"false","pipelines.cdcApplierStagingTablePartitionedByDate":"true","pipelines.changeDataFeed.enabled":"true","pipelines.checkImplicitDeletionReason":"true","pipelines.cleanupOrphanedBackingTables":"true","pipelines.copyFlowFuncResultWithDecomposition":"true","pipelines.createMvStEntryBeforeBackingTable":"true","pipelines.dataPlaneStructuredLogging.enabled":"true","pipelines.decomposition.enableOnSnapshotFlow":"true","pipelines.decomposition.enabled":"true","pipelines.disableClearingIncrementalOnVirtualize":"true","pipelines.disablePredictiveOptimization":"true","pipelines.disallowFlowExecutionRetryForSchemaChangeExceptionsInTriggeredMode":"true","pipelines.disallowTableMutationsDuringMaintenance":"true","pipelines.dltDebugger.enableNodeStatusReporter":"true","pipelines.dropAndRecreateBadBackingTables":"true","pipelines.dropInactiveTables":"false","pipelines.earlyStagingTableCleanupEnabled":"true","pipelines.emitDeprecationEventForCustomSchemaUsage":"true","pipelines.emitSinkDefinition":"true","pipelines.enableApplyChangeSCD1Materialization":"false","pipelines.enableApplyChangesLineage.enabled":"true","pipelines.enableBatchTokenOnUpdateId":"false","pipelines.enableBlockList

4. One use case for the event log is to examine data quality metrics for all runs of your pipeline. These metrics provide valuable insights into your pipeline, both in the short term and long term. Metrics are captured for each constraint throughout the entire lifetime of the table.

   Below is an example query to obtain those metrics. We won’t dive into the JSON parsing code here. This example simply demonstrates what’s possible with the **event_log**.

   Run the cell and observe the results. Notice the following:
   - The **passing_records** for each constraint are displayed.
   - The **failing_records** (WARN) for each constraint are displayed.

**NOTE:** If you have selected **Run pipeline with full table refresh** at any time during your pipeline, your results will include metrics from previous runs as well as from the full refresh. Additional logic is required to isolate results after the full table refresh. This is outside the scope of this course.


In [0]:
CREATE OR REPLACE TEMPORARY VIEW dq_source_vw AS
SELECT explode(
            from_json(details:flow_progress:data_quality:expectations,
                      "array<struct<name: string, dataset: string, passed_records: int, failed_records: int>>")
          ) AS row_expectations
   FROM default.event_log_demo_5
   WHERE event_type = 'flow_progress';


-- View the data
SELECT 
  row_expectations.dataset as dataset,
  row_expectations.name as expectation,
  SUM(row_expectations.passed_records) as passing_records,
  SUM(row_expectations.failed_records) as warnings_records
FROM dq_source_vw
GROUP BY row_expectations.dataset, row_expectations.name
ORDER BY dataset;

dataset,expectation,passing_records,warnings_records
labuser11058730_1754017152.2_silver_db.orders_silver_demo5,valid_date,272,0
labuser11058730_1754017152.2_silver_db.orders_silver_demo5,valid_id,272,0
labuser11058730_1754017152.2_silver_db.orders_silver_demo5,valid_notifications,272,0
labuser11058730_1754017152.2_silver_db.status_silver_demo5,valid_order_status,885,61
labuser11058730_1754017152.2_silver_db.status_silver_demo5,valid_timestamp,946,0


### Summary

This was a quick introduction to the pipeline **event_log**. With the **event_log**, you can investigate all aspects of your pipeline runs to explore the runs as well as create overall reports. Feel free to investigate the **event_log** further on your own.

## Additional Resources

- [Lakeflow Declarative Pipelines properties reference](https://docs.databricks.com/aws/en/dlt/properties#dlt-table-properties)

- [Table properties and table options](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-syntax-ddl-tblproperties)

- [Triggered vs. continuous pipeline mode](https://docs.databricks.com/aws/en/dlt/pipeline-mode)

- [Development and production modes](https://docs.databricks.com/aws/en/dlt/updates#development-and-production-modes)

- [Monitor Lakeflow Declarative Pipelines](https://docs.databricks.com/aws/en/dlt/observability)

- **Materialized views include built-in optimizations where applicable:**
  - [Incremental refresh for materialized views](https://docs.databricks.com/aws/en/optimizations/incremental-refresh)
  - [Delta Live Tables Announces New Capabilities and Performance Optimizations](https://www.databricks.com/blog/2022/06/29/delta-live-tables-announces-new-capabilities-and-performance-optimizations.html)
  - [Cost-effective, incremental ETL with serverless compute for Delta Live Tables pipelines](https://www.databricks.com/blog/cost-effective-incremental-etl-serverless-compute-delta-live-tables-pipelines)

- **Stateful joins:** For stateful joins in pipelines (i.e., joining incrementally as data is ingested), refer to the [Optimize stateful processing in Lakeflow Declarative Pipelines with watermarks](https://docs.databricks.com/aws/en/dlt/stateful-processing) documentation. **Stateful joins are an advanced topic and outside the scope of this course.**


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>